## 6. This Jupyter Notebook shows the sixth iteration. This iteration uses Social Mobility Rank as target and all social mobility features as features.

In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import r2_score


In [2]:
# Loading data
file_path = Path("Resources/2022-College-Guide-Main-Rankings-clear.csv")
df_rankings = pd.read_csv(file_path)
df_rankings.head()


,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank,Unnamed: 40
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,0.290903,10,NO,3,0.068361,401,3,2,187,NaN
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,0.160573,71,YES,6,0.246743,209,2,9,40,NaN
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,0.065477,332,NO,2,0.000000,436,5,1,324,NaN
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,0.064500,339,NO,6,0.000000,436,7,6,224,NaN
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,0.089523,224,YES,5,0.170164,305,12,15,20,NaN


In [3]:
df_rankings.columns

Index(['UnitID', 'Rank', 'Name', '8-year graduation rate',
       'Graduation rate rank',
       'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
       'Graduation rate performance rank', 'Pell/non-Pell graduation gap',
       'Pell graduation gap rank', 'Number of Pell graduates',
       'Actual vs. predicted Pell enrollment', 'Pell performance rank',
       'Median earnings 10 years after entering college',
       'Predicted median earnings 10 years after entering college',
       'Earnings performance rank',
       'Net price of attendance for families below $75,000 income',
       'Net price rank', '% of loan principal remaining 5 years later',
       'Repayment rank', 'Predicted principal remaining',
       'Repayment rate performance rank', 'Research expenditures, in millions',
       'Research expenditures rank', 'Bachelor's to PhD rank',
       'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
       'Faculty receiving sign

In [4]:
df_rankings.dtypes

UnitID                                                                            int64
Rank                                                                              int64
Name                                                                             object
8-year graduation rate                                                          float64
Graduation rate rank                                                              int64
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    float64
Graduation rate performance rank                                                  int64
Pell/non-Pell graduation gap                                                    float64
Pell graduation gap rank                                                          int64
Number of Pell graduates                                                        float64
Actual vs. predicted Pell enrollment                                            float64
Pell performance rank           

In [5]:
df_rankings.count()

UnitID                                                                          442
Rank                                                                            442
Name                                                                            442
8-year graduation rate                                                          442
Graduation rate rank                                                            442
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    442
Graduation rate performance rank                                                442
Pell/non-Pell graduation gap                                                    442
Pell graduation gap rank                                                        442
Number of Pell graduates                                                        442
Actual vs. predicted Pell enrollment                                            442
Pell performance rank                                                       

In [6]:
# Drop last column
df_rankings.drop('Unnamed: 40', axis=1, inplace=True)

In [7]:
# Encoding Carnegie column (0=NO, 1=YES)

le = LabelEncoder()
df2_rankings = df_rankings.copy()
df2_rankings['Earns Carnegie Community Engagement Classification?'] = le.fit_transform(df2_rankings['Earns Carnegie Community Engagement Classification?'])


In [8]:
df2_rankings.head()

,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [9]:
# Define the features set.
X = df2_rankings.copy()

X = X.drop(columns=['UnitID', 'Rank', 'Name','Research expenditures, in millions', 'Research expenditures rank',
    "Bachelor's to PhD rank", 'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
    'Faculty receiving significant awards', 'Faculty in National Academies', 'Faculty accolades rank',
    'AmeriCorps/Peace Corps rank',
    'ROTC rank',
    '% of federal work-study funds spent on service rank', '% of federal work-study funds spent on service',
    'Earns Carnegie Community Engagement Classification?', 'Voting engagement points',
    '% of grads with service-oriented majors', 'Service-oriented majors rank',
    'Social mobility rank', 'Research rank', 'Service rank'
    ])
X.head()


,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,Actual vs. predicted Pell enrollment,Pell performance rank,Median earnings 10 years after entering college,Predicted median earnings 10 years after entering college,Earnings performance rank,"Net price of attendance for families below $75,000 income",Net price rank,% of loan principal remaining 5 years later,Repayment rank,Predicted principal remaining,Repayment rate performance rank
0,0.963302,6,0.949654,187,-0.021703,46,260.6667,0.038096,122,97798,86131.63,15,1445.8400,2,0.322153,10,0.383026,46
1,0.964294,5,0.938620,149,-0.041917,104,376.6667,-0.000590,211,103246,82677.28,1,7368.1700,30,0.351408,13,0.401545,62
2,0.950971,10,0.990213,334,-0.032985,72,182.6667,0.076769,65,111222,95700.68,5,3290.8450,7,0.305619,3,0.344599,87
3,0.976861,2,0.947812,137,-0.052740,132,153.3333,0.062997,85,95689,82261.91,9,3672.0020,9,0.360991,18,0.388965,124
4,0.954474,9,0.967952,267,-0.014278,36,231.6667,0.028513,142,93115,85402.80,38,671.8563,1,0.361023,19,0.360074,222


In [10]:
# Checking datatypes to make sure there are no strings
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 18 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   8-year graduation rate                                                        442 non-null    float64
 1   Graduation rate rank                                                          442 non-null    int64  
 2   Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.  442 non-null    float64
 3   Graduation rate performance rank                                              442 non-null    int64  
 4   Pell/non-Pell graduation gap                                                  442 non-null    float64
 5   Pell graduation gap rank                                                      442 non-null    int64  
 6   Number of Pell graduates          

In [11]:
# Define the target set.
y = df2_rankings["Social mobility rank"].ravel()
y[:5]


array([ 3,  2,  5,  7, 12], dtype=int64)

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [13]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (331, 18)
Training Labels Shape: (331,)
Testing Features Shape: (111, 18)
Testing Labels Shape: (111,)


## Random Forest Regressor

In [14]:
# Instantiate model with 1000 decision trees
rf_model = RandomForestRegressor(n_estimators = 1500, random_state = 82)
# Train the model on training data
rf_model=rf_model.fit(X_train_scaled, y_train)

In [15]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)


In [16]:
# Evaluate the performance with r2 score
r2_rf= r2_score(y_test,rf_predictions)
print('The r2 score for this Random Forest Regressor model is:', r2_rf)

The r2 score for this Random Forest Regressor model is: 0.8571594509221567


In [17]:
# Getting accuracy data with Regressor/continuous variable model

# Calculate the absolute errors
errors = abs(rf_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Mean Absolute Error: 37.31
Accuracy: 59.88 %.


In [18]:
y_test

array([220,  43, 193, 364, 111, 359, 333, 221, 292,  48,  91,  72, 129,
       117, 131, 430,  84,  66,  16,  42, 324, 380, 356, 311, 171, 413,
       194, 293, 110, 126, 310, 218, 404, 227, 169, 150, 134, 431, 279,
       415, 156, 167, 278, 318, 209, 338, 123,  26, 270,  56, 187, 153,
        25, 188, 272,  31, 304, 301, 216,  14, 172,  65, 365,  58, 349,
       203, 241,  32,  28, 335,   5, 173, 177,  70, 148, 268, 313,  88,
       346, 374,  18, 408, 100, 101, 344, 395, 362, 334,  63, 163, 253,
        23, 281, 238, 189, 271,  39, 154, 410,  27,  99, 200,  49, 286,
       400, 245,  55, 406, 370, 254, 239], dtype=int64)

In [19]:
errors

array([1.48626667e+01, 4.21166667e+01, 5.50800000e+00, 4.81206667e+01,
       7.61866667e+00, 7.60206667e+01, 8.63000000e+00, 2.26160000e+01,
       1.86666667e-01, 3.22573333e+01, 7.50733333e+00, 7.65206667e+01,
       5.14186667e+01, 2.64893333e+01, 5.13666667e+00, 5.61306667e+01,
       1.94771333e+02, 2.36473333e+01, 3.18546667e+01, 1.47780000e+01,
       3.11200000e+01, 2.99006667e+01, 2.27773333e+01, 2.81453333e+01,
       6.80346667e+01, 1.48348667e+02, 1.32613333e+01, 2.67980000e+01,
       7.94686667e+01, 3.95960000e+01, 4.45020000e+01, 5.53533333e+00,
       5.74026667e+01, 3.81933333e+00, 2.40766667e+01, 4.04340000e+01,
       8.07253333e+01, 5.01506667e+01, 1.25060000e+01, 9.11953333e+01,
       5.90486667e+01, 5.12860000e+01, 1.43553333e+01, 6.58193333e+01,
       5.88400000e+00, 5.71600000e+01, 4.94286667e+01, 3.02386667e+01,
       1.40660000e+01, 8.67626667e+01, 2.23866667e+01, 3.90146667e+01,
       2.70500000e+01, 3.50400000e+01, 5.38246667e+01, 8.15840000e+01,
      

In [20]:
from sklearn import metrics
# https://machinelearningmastery.com/regression-metrics-for-machine-learning

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, rf_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_predictions)))

Mean Absolute Error: 37.31253453453454
Mean Squared Error: 2213.3633126686696
Root Mean Squared Error: 47.046395320669035


In [22]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([0.03567171, 0.03934173, 0.00468474, 0.13288836, 0.08406269,
       0.08447697, 0.0408013 , 0.01188363, 0.0116243 , 0.03201962,
       0.01101143, 0.10775726, 0.09433425, 0.08950855, 0.01903061,
       0.02170678, 0.0058367 , 0.17335937])

In [24]:
# Sort the features by their importance.
sorted_list=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted_df=pd.DataFrame(sorted_list, columns=['Importance', 'Feature'])
sorted_df

,Importance,Feature
0,0.173359,Repayment rate performance rank
1,0.132888,Graduation rate performance rank
2,0.107757,Earnings performance rank
3,0.094334,Net price of attendance for families below $75...
4,0.089509,Net price rank
5,0.084477,Pell graduation gap rank
6,0.084063,Pell/non-Pell graduation gap
7,0.040801,Number of Pell graduates
8,0.039342,Graduation rate rank
9,0.035672,8-year graduation rate


In [25]:
sorted_df.to_csv('Feature_Importances_with_SocMobRank_as_target.csv')  

## Support Vector Regression

In [26]:
# Create a SVM model

# Instantiate a SVM model
from sklearn.svm import SVR

# Fit the data
regressor = SVR(kernel = 'poly')
svm_model = regressor.fit(X_train, y_train)
svm_model

#Testing different kernels for improved r2 score
#rbf kernel r2 score = .103
#linear kernel r2 score = -209.628
#sigmoid kernel r2 = -.013
#poly(polynomial) kernel r2= .313


SVR(kernel='poly')

In [27]:
# Predicting the Test set results
svm_predictions = svm_model.predict(X_test)

In [28]:
from sklearn.metrics import r2_score
r2_svm=r2_score(y_test, svm_predictions)
print('The r2 score for this SVM model is:', r2_svm)

The r2 score for this SVM model is: 0.23188394999137107


## Decision Tree Regression

In [30]:
# Training the Decision Tree Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 0)
dt_model = dt_model.fit(X_train, y_train)
dt_model

DecisionTreeRegressor(random_state=0)

In [31]:
# Predicting the Test set results
dt_predictions = dt_model.predict(X_test)

In [32]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_dt= r2_score(y_test,dt_predictions)
print('The r2 score for this Decision Tree model is:', r2_dt)

The r2 score for this Decision Tree model is: 0.6150148177133232


## Multiple Linear Regression

In [33]:
# Training the multiple regression model on the Training set
from sklearn.linear_model import LinearRegression
mlr_model = LinearRegression()
mlr_model = mlr_model.fit(X_train, y_train)
mlr_model

LinearRegression()

In [34]:
# Predicting the Test set results
mlr_predictions = mlr_model.predict(X_test)

In [35]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_mlr= r2_score(y_test,mlr_predictions)
print('The r2 score for this Multiple Linear Regression model is:', r2_mlr)

The r2 score for this Multiple Linear Regression model is: 0.9581228731179956


In [36]:
# get importance
importance = mlr_model.coef_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: -328.57378
Feature: 1, Score: 0.11738
Feature: 2, Score: 315.31452
Feature: 3, Score: 0.05202
Feature: 4, Score: -367.20422
Feature: 5, Score: 0.10504
Feature: 6, Score: -0.01944
Feature: 7, Score: -109.98909
Feature: 8, Score: 0.07468
Feature: 9, Score: 0.00054
Feature: 10, Score: -0.00072
Feature: 11, Score: 0.30742
Feature: 12, Score: 0.00488
Feature: 13, Score: 0.10504
Feature: 14, Score: 53.29900
Feature: 15, Score: 0.05053
Feature: 16, Score: 78.93328
Feature: 17, Score: 0.19714
